In [1]:
import numpy as np
import pandas as pd
import torch
import nltk
import os.path

from tqdm.autonotebook import tqdm, trange
from sentence_transformers import SentenceTransformer 

/var/folders/pz/n_v30dyj09bfnyhk_h1dnfnm0000gn/T/ipykernel_73586/3444334441.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/opt/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 6): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: /opt/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in: /opt/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib
 in /opt/anaconda3/lib/python3.11/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvis

In [2]:
sentence_model = SentenceTransformer("thenlper/gte-large")

def get_sentence_embedding(text):
    if isinstance(text,str):
        if not text.strip(): 
            print("Attempted to get embedding for empty text.")
            return []
        
        return sentence_model.encode(text)
    
    elif isinstance(text,list):
        return [get_sentence_embedding(x) for x in text]
        

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [4]:
text = "This is a fake review."
text2 = "This is not a fake review."

embedding = get_sentence_embedding(text)
print(type(embedding))

<class 'numpy.ndarray'>
[-0.00613885 -0.01058692  0.00132796 ... -0.02228644 -0.00615896
 -0.02127945]


In [5]:
embedding

array([-0.00613885, -0.01058692,  0.00132796, ..., -0.02228644,
       -0.00615896, -0.02127945], dtype=float32)

In [6]:
a = [text, text2]
get_sentence_embedding(a)

[array([-0.00613885, -0.01058692,  0.00132796, ..., -0.02228644,
        -0.00615896, -0.02127945], dtype=float32),
 array([ 0.00717232,  0.00192557, -0.01863513, ..., -0.01114191,
         0.01110455, -0.00416858], dtype=float32)]

In [7]:
df = pd.read_csv("../raw data/combined_data.csv")

In [8]:
df.head()

,Text,Label,Model,Original dataset,Row in original dataset
0,While driverless cars present many promising b...,Machine,darragh_claude_v7,essays,13355
1,Homework Clubs: The Key to Unlocking Academic ...,Machine,llama2_chat,essays,7249
2,"""The legalization of marijuana has been a cont...",Machine,falcon_180b_v1,essays,2603
3,Taking the opportunity to learn new things can...,Machine,mistral7binstruct_v1,essays,3993
4,Working with a partner is an effective way fo...,Machine,mistral7binstruct_v2,essays,3773


In [9]:
tqdm.pandas()
embeddings = df['Text'].progress_apply(get_sentence_embedding)

  0%|          | 0/80000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
df.head()

,Text,Label,Original dataset,Row in original dataset,embedding
0,The idea of graduating high school in three ye...,Machine,essays,26613,"[0.01476596, -0.013095475, 0.002932728, -0.011..."
1,"Hey, I'm so excited to write this essay about ...",Machine,essays,26326,"[0.00292786, -0.013083563, 0.0047025573, -0.00..."
2,Introduction\n\nSelf-reliance is a concept tha...,Machine,essays,30579,"[-0.015756093, -0.023221416, -0.010959062, -0...."
3,"Sure, here's my attempt at writing an essay as...",Machine,essays,33547,"[0.013002162, 0.011017485, -0.03551094, 0.0290..."
4,The legalization of marijuana is a highly deba...,Machine,essays,33768,"[0.0016188276, 0.011155421, -0.004596148, 0.00..."


In [13]:
df.to_csv('combined_data_with_embeddings.csv')

In [18]:
df.to_parquet('combined_data_with_embeddings.parquet')

In [14]:
embeddings = df['embedding']

In [15]:
df['embedding'].apply(lambda a : len(a))

0        1024
1        1024
2        1024
3        1024
4        1024
         ... 
79995    1024
79996    1024
79997    1024
79998    1024
79999    1024
Name: embedding, Length: 80000, dtype: int64

In [16]:
print(sentence_model.max_seq_length)

512
